In [1]:
# IMPORTS
from ipywidgets import HBox, VBox, Label, Layout, AppLayout, Output, Tab, Text
from IPython.display import display
import sys, os
import yaml
from pprint import pprint

# append parent of cwd to sys path to find modules
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

# surpress pygame version prompt
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"

# enable matplotlib widgets
%matplotlib widget

from src.app_batch import AppBatch as App
from src.io.plot import multi_evaluation_bars, two_multibar_plots, animated_plotly_pianoroll, multitrack_plotly_pianoroll
from src.io.output import play_button
from src.io.input import loadMidiFile

# widget objects
from dashboards.widgets import inputHeading, inputSelect, inputUpload, generationHeading, checkpointSelect, tempSlider, generationInfo, adaptationHeading, stepsSelect, startButton, applyGenerationSettingsButton, applyAdaptationSettingsButton, generationAmount, paginationSlider, iconButton

# widget functions
from dashboards.widgets import h2_heading

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
##########################
#C#    OUTPUT & APP    ###
##########################

log = Output()

summary_container = Output()
pianoroll_container = Output()
play_button_container = Output()
evaluation_container = Output()
meta_container = Output()
generation_info = Output()

# init
app = App(log=log)

with generation_info:
    display(generationInfo)

In [3]:
##########################
#M#   SELECTION DATA   ###
##########################

adaptation_operations = app.get_adaptation_operations()
generators = app.get_generators()
demo_melodies = app.get_demo_input()

checkpointSelect.options = [(g[0], generators.index(g)) for g in generators]
stepsSelect.options = adaptation_operations
inputSelect.options = [(d[0], demo_melodies.index(d)) for d in demo_melodies]

generation_page_slider = paginationSlider(generationAmount.value, 'Generation')

In [4]:
##########################
#C#     CONTROLLER     ###
##########################

def apply_generator_settings(b):
    generator = app.apply_generator_settings(checkpointSelect.value, tempSlider.value)
    generation_info.clear_output()
    with generation_info:
        print('Using: ' + generator)

def apply_adaptation_settings(b):
    app.apply_adaptation_settings(stepsSelect.value)

result = None

def run_adaptation(b):
    # reset output containers
    summary_container.clear_output()

    # TODO handle file upload
    input_path = demo_melodies[inputSelect.value][1]
    result = app.run(input_path, generationAmount.value, False)
    print('\n### RESULT ###')
    pprint(result)

    # init pagination
    generation_page_slider.max = generationAmount.value-1

    with summary_container:
        # TODO print metrics ? 
        pass


    # avg plots
    sim_data = [result['generation_avg_similarity']['normalized'], result['adaptation_avg_similarity']['normalized']]
    var_data = [result['generation_variance']['normalized'], result['adaptation_variance']['normalized']]

    two_multibar_plots(sim_data, ['Generations', 'Adaptations'], 'Similarity', var_data, ['Generations', 'Adaptations'], 'Variance', 'Average over Generation-Adaptation Pairs', summary_container)

    display_details(None)


def display_details(b):
    # reset output containers
    pianoroll_container.clear_output()
    play_button_container.clear_output()
    evaluation_container.clear_output()
    meta_container.clear_output()

    if app.result is None:
        with meta_container:
            print('ERROR: No results available.')
            return
    
    cr_set = app.result['generations'][generation_page_slider.value]

    # create pianoroll plots from results and print them
    midi_list = [cr_set.input_sequence.sequence, cr_set.generated_base_sequence.sequence, cr_set.output_sequence.sequence]
    name_list = ['Input', 'Generation', 'Result']

    multitrack_plotly_pianoroll(midi_list, name_list, 'Sequences', pianoroll_container)


    # display play buttons for audio
    with play_button_container: 
        display(h2_heading('Input:'))
    play_button(midi_list[0], play_button_container)
    with play_button_container: 
        display(h2_heading('Generation:'))    
    play_button(midi_list[1], play_button_container)
    with play_button_container: 
        display(h2_heading('Adaptation:'))
    play_button(midi_list[2], play_button_container)


    # create evaluation plots
    eval_data = [cr_set.generation_similarity['normalized'], cr_set.output_similarity['normalized']]
    multi_evaluation_bars(eval_data, evaluation_container, ['Generated Sequence', 'Adapted Sequence'])


     # create animated adaptation steps plot
    steps_midi_list = [step['intermediate_result'] for step in cr_set.output_sequence.meta['adaptation']['steps']]
    steps_names = [step['name'] for step in cr_set.output_sequence.meta['adaptation']['steps']]
    steps_midi_list.insert(0, cr_set.generated_base_sequence.sequence)
    steps_names.insert(0, 'Original Generation')

    pitches = []
    for seq in steps_midi_list:
        pitches.extend([note.pitch for note in seq.instruments[0].notes])
    
    animated_plotly_pianoroll(steps_midi_list, steps_names, 'Effect of Single Adaptation Steps', evaluation_container, [min(pitches)-1, max(pitches)+1])


    # print meta data
    with meta_container:
        display(h2_heading('Input Analysis:'))
        pprint(cr_set.input_sequence.analysis)
        display(h2_heading('Generation Analysis:'))
        pprint(cr_set.output_sequence.analysis)
        display(h2_heading('Process Meta Data:'))
        pprint(cr_set.output_sequence.meta)

def reload_details(change):
    display_details()


# settings button click handlers
startButton.on_click(run_adaptation)
applyGenerationSettingsButton.on_click(apply_generator_settings)
applyAdaptationSettingsButton.on_click(apply_adaptation_settings)

# pagination event handlers
refreshButton.on_click(display_details)
generation_page_slider.observe(reload_details, names='value')

In [5]:
##########################
#V#  SETTINGS SIDEBAR  ###
##########################

side_bar_layout = Layout(border='1px solid grey', padding='10px')

settings_box = VBox([
        generationHeading,
        HBox([Label('Model & Checkpoint:'), checkpointSelect]),
        HBox([Label('Temperature:'), tempSlider]),
        generation_info,
        applyGenerationSettingsButton,
        adaptationHeading,
        HBox([Label('Steps:'), stepsSelect]),
        applyAdaptationSettingsButton,
        inputHeading,
        inputSelect,
        HBox([Label('Or upload own file:'), inputUpload]),
        HBox([Label('Amount of generations:'), generationAmount]),
        startButton,
    ], layout = side_bar_layout)

In [6]:
##########################
#V#    RESULTS AREA    ###
##########################

main_layout = Layout(padding='10px')

# AVG & SUMMARY
summary_box = summary_container

# DETAILS
pagination_box = HBox([generation_page_slider, refreshButton])
pianorolls = HBox([pianoroll_container, play_button_container])
details_box = VBox([pagination_box, pianorolls, evaluation_container, meta_container], layout=main_layout)

In [7]:
##########################
#V#        TABS        ###
##########################

tabs = Tab()
tabs.children = [summary_box, details_box]
tabs.set_title(0, "Summary")
tabs.set_title(1, "Detailed Results")

In [8]:
##########################
#V#     APP LAYOUT     ###
##########################

AppLayout(left_sidebar=settings_box,
          center=tabs,
          header=None,
          footer=log,
          right_sidebar=None,
          pane_widths=[1,2,0],
          pane_heights=[0,4,1])

AppLayout(children=(Output(layout=Layout(grid_area='footer')), VBox(children=(HTML(value='<h2>Generation Setti…

In [9]:
# display_details(None)

In [11]:
# run_adaptation(startButton)

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

C:\Users\Eric\miniconda3\envs\mpro\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars




### RESULT ###
{'adaptation_avg_similarity': {'absolute': {'avg_ioi': 0.19058035714285718,
                                            'avg_pitch_interval': 1.9335134310134312,
                                            'note_count': 6.0,
                                            'note_length_histogram': 0.8791884106185958,
                                            'note_length_transition_matrix': 8.26272484244501,
                                            'pitch_class_histogram': 0.5688735621878585,
                                            'pitch_class_transition_matrix': 3.760255866122757,
                                            'pitch_count': 1.9,
                                            'pitch_range': 3.9},
                               'normalized': {'avg_ioi': 4.7393986466616855,
                                              'avg_pitch_interval': 4.833783577533579,
                                              'note_count': 3.0,
                                

HTML(value='<h2>Input:<h2>', placeholder='Input:')